In [2]:
import numpy as np
import quantecon as qe

In [84]:
# Calibrate transition matrix

x = np.linspace(.6,.9,100)

initial_state = np.array([0.95, 0.05])
for i in range(100):
    T = np.array([[0.9, 0.1],
                  [x[i], 1-x[i]]])
    T_power = np.power(T, 1000)
    new_state = np.dot(initial_state, T_power)
    print(new_state)
    
#     print(T_power)

#     diff = np.abs(new_state[0] - 0.05)
#     if diff < 1e-5:
#         print(diff)
#         print(x[i])

[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047769e-46 0.00000000e+00]
[1.66047

In [87]:
T = np.array([[0.9, 0.1],
              [1, 8.1]])
np.power(T, 1000)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


array([[1.74787125e-46, 0.00000000e+00],
       [1.00000000e+00,            inf]])